In [1]:
import configparser

In [2]:
config = configparser.ConfigParser()
config.read("C:/Users/chris/Documents/UVA/DS_5001/env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [3]:
source_files = f'{data_home}/gutenberg'
# data_prefix = 'eliot'

In [4]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [5]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk

In [6]:
import sys
sys.path.append(local_lib)

In [7]:
from textparser import TextParser

## Inspect

Since Project Gutenberg texts vary widely in their markup, we define our chunking patterns by hand.

In [8]:
# roman = '[IVXLCM]+'
# caps = "[A-Z';, -]+"
clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
# ohco_pat_list = [
#     (145,   rf"^\s*CHAPTER\s+{roman}.\s*$"),
#     (507,   rf"^\s*Chapter\s+{roman}\s*$"),
#     (6688,   rf"^\s*Chapter\s+{roman}.\s*$"),
# ]

# All are 'chap'and 'm'
# ohco_pat_list = [
#     (16,   r"^Chapter\s+[IVXLC]+\.\s*\n[A-Z ,'\-“’]+(\n\s*){2}"),         # Peter Pan (BARRIE_JM_PETER_PAN-pg16.txt)
#     (55,   r"^Chapter\s+[IVXLC]+\.\s+.*$"),         # Wizard of Oz (BAUM_FRANK_THE_WONDERFUL_WIZARD_OF_OZ-pg55.txt)
#     (11,   r"^CHAPTER\s+[IVXLC]+\.\s+.*$"),            # Alice in Wonderland (CARROLL_LEWIS_ALICES_ADVENTURES_IN_WONDERLAND-pg11.txt)
#     (500,  r"^Chapter\s+[IVXLC]+\s+.*$"),            # Pinocchio (COLLODI_CARLO_THE_ADVENTURES_OF_PINOCCHIO-pg500.txt)
#     (24022,r"^STAVE\s+[IVXLC]+\s+—\s+.*$"),          # Christmas Carol (DICKENS_CHARLES_A_CHRISTMAS_CAROL-pg24022.txt)
#     (501,  r"^CHAPTER\s+[0-9IVXLC]+\s+.*$"),         # Doctor Dolittle (LOFTING_HUGH_THE_STORY_OF_DOCTOR_DOLITTLE-pg501.txt)
#     (67098,r"^Chapter\s+[IVXLC]+\s+.*$"),            # Winnie-the-Pooh (MILNE_AA_WINNIE_THE_POOH-pg67098.txt)
#     (17253,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Dewey and Other Naval Commanders (ELLIS_SYLVESTER_EDWARD_DEWEY_AND_OTHER_NAVAL_COMMANDERS-pg17253.txt)
#     (22352,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Life of Theodore Roosevelt (STRATMEYER_EDWARD_AMERICAN_BOYS_LIFE_OF_THEODORE_ROOSEVELT-pg22352.txt)
#     (22925,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Young Abraham Lincoln (WHIPPLE_WAYNE_THE_STORY_OF_YOUNG_ABRAHAM_LINCOLN-pg22925.txt)
#     (24730,r"^[A-Z\s']{5,}$"),                       # Daniel Boone (WILKIE_ELLIOT_KATHARINE_DANIEL_BOONE-pg24730.txt), wide all-caps chapter headings like "MOVING ON"
# ]

# ohco_pat_list = [
#     (16,   r"^Chapter\s+[IVXLC]+\.\s+.*$"),         # Peter Pan (BARRIE_JM_PETER_PAN-pg16.txt)
#     (55,   r"^\s*Chapter\s+[IVXLC]+\s*$"),         # Wizard of Oz (BAUM_FRANK_THE_WONDERFUL_WIZARD_OF_OZ-pg55.txt)
#     (11,   r"^CHAPTER\s+[IVXLC]+\.\s+.*$"),            # Alice in Wonderland (CARROLL_LEWIS_ALICES_ADVENTURES_IN_WONDERLAND-pg11.txt)
#     (500,  r"^Chapter\s+[IVXLC]+\s+.*$"),            # Pinocchio (COLLODI_CARLO_THE_ADVENTURES_OF_PINOCCHIO-pg500.txt)
#     (24022,r"^STAVE\s+[IVXLC]+\s+—\s+.*$"),          # Christmas Carol (DICKENS_CHARLES_A_CHRISTMAS_CAROL-pg24022.txt)
#     (501,  r"^CHAPTER\s+[0-9IVXLC]+\s+.*$"),         # Doctor Dolittle (LOFTING_HUGH_THE_STORY_OF_DOCTOR_DOLITTLE-pg501.txt)
#     (67098,r"^Chapter\s+[IVXLC]+\s+.*$"),            # Winnie-the-Pooh (MILNE_AA_WINNIE_THE_POOH-pg67098.txt)
#     (17253,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Dewey and Other Naval Commanders (ELLIS_SYLVESTER_EDWARD_DEWEY_AND_OTHER_NAVAL_COMMANDERS-pg17253.txt)
#     (22352,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Life of Theodore Roosevelt (STRATMEYER_EDWARD_AMERICAN_BOYS_LIFE_OF_THEODORE_ROOSEVELT-pg22352.txt)
#     (22925,r"^CHAPTER\s+[IVXLC]+\s+.*$"),            # Young Abraham Lincoln (WHIPPLE_WAYNE_THE_STORY_OF_YOUNG_ABRAHAM_LINCOLN-pg22925.txt)
#     (24730,r"^[A-Z\s']{5,}$"),                       # Daniel Boone (WILKIE_ELLIOT_KATHARINE_DANIEL_BOONE-pg24730.txt), wide all-caps chapter headings like "MOVING ON"
# ]

ohco_pat_list = [
    (41445,   rf"^\s*CHAPTER\s+{roman}.\s*$"),         # frankenstein
    (15238,   r'^(PREFACE|CHAPTER\s+[IVXLCDM]+\.?)'),         # mathilda
    (3420,   r"^CHAPTER\s+[0-9]+\.\s*$"),            # vindication of the rights of woman
    (134,  r"^CHAPTER\s+[0-9]+\s*$"),            # maria or the wrongs of woman
    (16357,  r"^CHAP\.\s+[IVXLC]+\.\s*$"),          # mary a fiction
    (18247,  rf"^\s*CHAPTER\s+{roman}.\s*$"),         # the last man
    (3529,  r'^(INTRODUCTION\.|LETTER [IVXLCDM]+\.)')            # letters written
]

## Register

We get each file and add to a library `LIB`.

In [9]:
# source_file_list = sorted(glob(f"{source_files}/*.*"))
# source_file_list = [s.replace('\\', '/').split('fp_data/',1)[-1] for s in sorted(glob(f"{source_files}/*.txt"))]
source_file_list = [s.replace('\\', '/') for s in sorted(glob(f"{source_files}/*.txt"))]



In [10]:
source_file_list

['C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_A_VINDICATION_OF_THE_RIGHTS_OF_WOMAN-pg3420.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_FRANKENSTEIN-pg41445.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_LETTERS_WRITTEN_DURING_A_SHORT_RESIDENCE_IN_SWEDEN_NORWAY_AND_DENMARK-pg3529.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MARIA_OR_THE_WRONGS_OF_WOMAN-pg134.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MARY_A_FICTION-pg16357.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MATHILDA-pg15238.txt',
 'C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_THE_LAST_MAN-pg18247.txt']

In [11]:
book_data = []
for source_file_path in source_file_list:
    book_id = int(source_file_path.split('-')[-1].split('.')[0].replace('pg',''))
    book_title = source_file_path.split('/')[-1].split('-')[0].replace('_', ' ')
    book_data.append((book_id, source_file_path, book_title))

In [12]:
LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()

In [13]:
LIB

,source_file_path,raw_title
book_id,,
134,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY MARIA OR THE WRONGS OF WOMAN
3420,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY A VINDICATION OF THE RIGHTS OF WOMAN
3529,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY LETTERS WRITTEN DURING A SHORT RE...
15238,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY MATHILDA
16357,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY MARY A FICTION
18247,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY THE LAST MAN
41445,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,SHELLEY MARY FRANKENSTEIN


In [14]:
try:
    LIB['author'] = LIB.raw_title.apply(lambda x: ', '.join(x.split()[:2]))
    LIB['title'] = LIB.raw_title.apply(lambda x: ' '.join(x.split()[2:]))
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass

In [15]:
LIB

,source_file_path,author,title
book_id,,,
134,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARIA OR THE WRONGS OF WOMAN
3420,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",A VINDICATION OF THE RIGHTS OF WOMAN
3529,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",LETTERS WRITTEN DURING A SHORT RESIDENCE IN SW...
15238,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MATHILDA
16357,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARY A FICTION
18247,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",THE LAST MAN
41445,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",FRANKENSTEIN


In [16]:
# # add publication year
# year_map = {'ALICES ADVENTURES IN WONDERLAND': '1865', 'PETER PAN': '1953', 'THE WONDERFUL WIZARD OF OZ': '1900', 'THE ADVENTURES OF PINOCCHIO': '1882', 
#            'THE STORY OF DOCTOR DOLITTLE': '1920', 'DEWEY AND OTHER NAVAL COMMANDERS': '1899', 'AMERICAN BOYS LIFE OF THEODORE ROOSEVELT': '1904', 'THE STORY OF YOUNG ABRAHAM LINCOLN': '1915',
#            'A CHRISTMAS CAROL': '1843', 'DANIEL BOONE': '1939', 'WINNIE THE POOH': '1926'}

# LIB['year'] = LIB['title'].map(year_map)

In [17]:
# # add title + pub year label
# LIB['label'] = LIB.title.str[:30] + ' (' + LIB.year.astype('str') + ')'
# LIB['label'] = LIB.label.str.title()

In [18]:
LIB

,source_file_path,author,title
book_id,,,
134,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARIA OR THE WRONGS OF WOMAN
3420,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",A VINDICATION OF THE RIGHTS OF WOMAN
3529,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",LETTERS WRITTEN DURING A SHORT RESIDENCE IN SW...
15238,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MATHILDA
16357,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARY A FICTION
18247,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",THE LAST MAN
41445,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",FRANKENSTEIN


In [19]:
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))

In [20]:
LIB

,source_file_path,author,title,chap_regex
book_id,,,,
134,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARIA OR THE WRONGS OF WOMAN,^CHAPTER\s+[0-9]+\s*$
3420,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",A VINDICATION OF THE RIGHTS OF WOMAN,^CHAPTER\s+[0-9]+\.\s*$
3529,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",LETTERS WRITTEN DURING A SHORT RESIDENCE IN SW...,^(INTRODUCTION\.|LETTER [IVXLCDM]+\.)
15238,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MATHILDA,^(PREFACE|CHAPTER\s+[IVXLCDM]+\.?)
16357,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",MARY A FICTION,^CHAP\.\s+[IVXLC]+\.\s*$
18247,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",THE LAST MAN,^\s*CHAPTER\s+[IVXLCM]+.\s*$
41445,C:/Users/chris/Documents/UVA/DS_5001/fp_data/g...,"SHELLEY, MARY",FRANKENSTEIN,^\s*CHAPTER\s+[IVXLCM]+.\s*$


In [21]:
# def tokenize_collection(LIB):

#     clip_pats = [
#         r"\*\*\*\s*START OF",
#         r"\*\*\*\s*END OF"
#     ]

#     books = []
#     for book_id in LIB.index:

#         # Announce
#         print("Tokenizing", book_id, LIB.loc[book_id].title)

#         # Define vars
#         # chap_regex = LIB.loc[book_id].chap_regex
#         # ohco_pats = [('chap', chap_regex, 'm')]
#         # src_file_path = LIB.loc[book_id].source_file_path
#         chap_regex = LIB.loc[book_id].chap_regex
#         ohco_pats = [
#             ('chap', chap_regex, 'm'), 
#             ('para', r'\n{2,}', 'd'), 
#             ('sent', None, 'nltk'), 
#             ('token', None, 'nltk')
#         ]
#         src_file_path = LIB.loc[book_id].source_file_path

#         # Create object
#         text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
#         # text = TextImporter(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats) 

#         # Define parameters
#         text.verbose = True
#         text.strip_hyphens = True
#         text.strip_whitespace = True

#         # Parse
#         text.import_source().parse_tokens();

#         # Name things
#         text.TOKENS['book_id'] = book_id
#         text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)
#         # text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

#         # Add to list
#         books.append(text.TOKENS)
        
#     # Combine into a single dataframe
#     CORPUS = pd.concat(books).sort_index()

#     # Clean up
#     del(books)
#     del(text)
        
#     print("Done")
        
#     return CORPUS

In [22]:
def tokenize_collection(LIB):
    books = []
    for book_id in LIB.index:
        # Announcing the current working book
        print("Tokenizing", book_id, LIB.loc[book_id].title)
        # Deriving vars from LIB Model
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path
        # Creating TextParser object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True
        # Parse
        text.import_source().parse_tokens();
        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)
        # Add to list
        books.append(text.TOKENS)
        print("\n\n\n")
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()
    # Clean up
    del(books)
    del(text)
    print("Done")
    return CORPUS

In [23]:
# !pip install nltk

In [24]:
CORPUS = tokenize_collection(LIB)

Tokenizing 134 MARIA OR THE WRONGS OF WOMAN
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MARIA_OR_THE_WRONGS_OF_WOMAN-pg134.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^CHAPTER\s+[0-9]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model






Tokenizing 3420 A VINDICATION OF THE RIGHTS OF WOMAN
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_A_VINDICATION_OF_THE_RIGHTS_OF_WOMAN-pg3420.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^CHAPTER\s+[0-9]+\.\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model




Tokenizing 3529 LETTERS WRITTEN DURING A SHORT RESIDENCE IN SWEDEN NORWAY AND DENMARK
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_LETTERS_WRITTEN_DURING_A_SHORT_RESIDENCE_IN_SWEDEN_NORWAY_AND_DENMARK-pg3529.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(INTRODUCTION\.|LETTER [IVXLCDM]+\.)
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model


C:\Users/chris/Documents/UVA/DS_5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Parsing OHCO level 3 token_num by NLTK model




Tokenizing 15238 MATHILDA
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MATHILDA-pg15238.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(PREFACE|CHAPTER\s+[IVXLCDM]+\.?)
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model


C:\Users/chris/Documents/UVA/DS_5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Parsing OHCO level 3 token_num by NLTK model




Tokenizing 16357 MARY A FICTION
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_MARY_A_FICTION-pg16357.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^CHAP\.\s+[IVXLC]+\.\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model




Tokenizing 18247 THE LAST MAN
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_THE_LAST_MAN-pg18247.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*CHAPTER\s+[IVXLCM]+.\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model




Tokenizing 41445 FRANKENSTEIN
Importing  C:/Users/chris/Documents/UVA/DS_5001/fp_data/gutenberg/SHELLEY_MARY_FRANKENSTEIN